In [1]:
# !pip install "git+https://github.com/nbukhanchenko/trans-timegrad"
# !pip install --upgrade --force-reinstall gluonts==0.14.3
# !pip install --upgrade --force-reinstall seaborn==0.13.1
# !pip install --upgrade --force-reinstall pandas==2.2.0
# !pip install --upgrade --force-reinstall numpy==1.23.5
# !pip install --upgrade --force-reinstall lightning==2.1.3
# !pip install --upgrade --force-reinstall diffusers==0.25.1

  Cloning https://github.com/nbukhanchenko/pytorch-ts to /tmp/pip-req-build-v_ulhyju
  Running command git clone --filter=blob:none --quiet https://github.com/nbukhanchenko/pytorch-ts /tmp/pip-req-build-v_ulhyju
  Resolved https://github.com/nbukhanchenko/pytorch-ts to commit 0c2dc556629b6bd772570a3201389f2ec657482c
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.1 MB/s eta 0:00:00
  Created wheel for pytorchts: filename=pytorchts-0.7.0-py3-none-any.whl size=124397 sha256=9271eaf60e875204cc79dc56a5bd72423757a6e92e85f02d471f07da6dddc03d
  Stored in directory: /tmp/pip-ephem-wheel-cache-h5njkwj8/wheels/72/4a/96/811c880612ee9a2607eef4d77cf69e8361df34c3c3ab80afdd
Successfully built pytorchts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━

In [6]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch

In [7]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator
from diffusers import (
    DDPMScheduler,
    PNDMScheduler,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
    KDPM2DiscreteScheduler,
    DEISMultistepScheduler,
)

from pts.model.time_grad import TimeGradEstimator
from pts.dataset.repository.datasets import dataset_recipes

In [8]:
def alpha_for_percentile(p):
    return (p / 100.0) ** 0.3

def plot(
    target,
    forecast,
    prediction_length,
    prediction_intervals=(50.0, 90.0),
    color="g",
    fname=None,
):
    label_prefix = ""
    rows = 4
    cols = 4
    fig, axs = plt.subplots(rows, cols, figsize=(24, 24))
    axx = axs.ravel()
    seq_len, target_dim = target.shape

    ps = [50.0] + [
        50.0 + f * c / 2.0 for c in prediction_intervals for f in [-1.0, +1.0]
    ]

    percentiles_sorted = sorted(set(ps))

    for dim in range(0, min(rows * cols, target_dim)):
        ax = axx[dim]

        target[-2 * prediction_length :][dim].plot(ax=ax)

        ps_data = [forecast.quantile(p / 100.0)[:, dim] for p in percentiles_sorted]
        i_p50 = len(percentiles_sorted) // 2

        p50_data = ps_data[i_p50]
        p50_series = pd.Series(data=p50_data, index=forecast.index)
        p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)

        for i in range(len(percentiles_sorted) // 2):
            ptile = percentiles_sorted[i]
            alpha = alpha_for_percentile(ptile)
            ax.fill_between(
                forecast.index,
                ps_data[i],
                ps_data[-i - 1],
                facecolor=color,
                alpha=alpha,
                interpolate=True,
            )
            # Hack to create labels for the error intervals.
            # Doesn't actually plot anything, because we only pass a single data point
            pd.Series(data=p50_data[:1], index=forecast.index[:1]).plot(
                color=color,
                alpha=alpha,
                linewidth=10,
                label=f"{label_prefix}{100 - ptile * 2}%",
                ax=ax,
            )

    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]
    axx[0].legend(legend, loc="upper left")

    if fname is not None:
        plt.savefig(fname, bbox_inches="tight", pad_inches=0.05)

def prepare_dataset(dataset_name):
    dataset = get_dataset(dataset_name, regenerate=False)

    train_grouper = MultivariateGrouper(
        max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality)
    )
    test_grouper = MultivariateGrouper(
        num_test_dates=int(len(dataset.test) / len(dataset.train)),
        max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    )

    return {
        "train": train_grouper(dataset.train),
        "test": test_grouper(dataset.test),
        "metadata": dataset.metadata
    }

def prepare_predictor(dataset, max_epochs=256,
                      num_train_timesteps=150, beta_start=1e-4, beta_end=0.1, beta_schedule="linear",
                      context_length_coef=3, num_layers=2, hidden_size=64, lr=3e-4, weight_decay=1e-8, dropout_rate=0.1,
                      lags_seq=[1], num_inference_steps=149, batch_size=64, num_batches_per_epoch=64):
#     scheduler = PNDMScheduler(
#         num_train_timesteps=num_train_timesteps,
#         beta_start=beta_start,
#         beta_end=beta_end,
#         beta_schedule=beta_schedule,
#     )

#     scheduler = DDPMScheduler(
#         num_train_timesteps=num_train_timesteps,
#         beta_start=beta_start,
#         beta_end=beta_end,
#         beta_schedule=beta_schedule,
#     )

    scheduler = DEISMultistepScheduler(
        num_train_timesteps=num_train_timesteps,
        beta_start=beta_start,
        beta_end=beta_end,
        beta_schedule=beta_schedule,
    )

    estimator = TimeGradEstimator(
        freq=dataset["metadata"].freq,
        prediction_length=dataset["metadata"].prediction_length,
        input_size=int(dataset["metadata"].feat_static_cat[0].cardinality),
        scheduler=scheduler,
        context_length=dataset["metadata"].prediction_length * context_length_coef,
        num_layers=num_layers,
        hidden_size=hidden_size,
        lr=lr,
        weight_decay=weight_decay,
        dropout_rate=dropout_rate,
        scaling="mean",
        lags_seq=lags_seq,
        num_inference_steps=num_inference_steps,
        batch_size=batch_size,
        num_batches_per_epoch=num_batches_per_epoch,
        trainer_kwargs=dict(max_epochs=max_epochs, accelerator="gpu", devices="1"),
    )

    return estimator.train(dataset["train"], cache_data=True, shuffle_buffer_length=1024)

def prepare_metrics(dataset, predictor, num_samples=100):
    evaluator = MultivariateEvaluator(
        quantiles=(np.arange(20) / 20.0)[1:], target_agg_funcs={"sum": np.sum}
    )

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset["test"], predictor=predictor, num_samples=num_samples
    )
    forecasts = list(forecast_it)
    targets = list(ts_it)
    agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset["test"]))

    return forecasts, targets, agg_metric

def prepare_statistics(dataset, forecasts, targets, agg_metric, precision=3):
    print("CRPS: {}".format(round(agg_metric["mean_wQuantileLoss"], precision)))
    print("ND: {}".format(round(agg_metric["ND"], precision)))
    print("NRMSE: {}".format(round(agg_metric["NRMSE"], precision)))
    print("MSE: {}".format(round(agg_metric["MSE"], precision)))

    print("-" * 32)

    print("CRPS-Sum: {}".format(round(agg_metric["m_sum_mean_wQuantileLoss"], precision)))
    print("ND-Sum: {}".format(round(agg_metric["m_sum_ND"], precision)))
    print("NRMSE-Sum: {}".format(round(agg_metric["m_sum_NRMSE"], precision)))
    print("MSE-Sum: {}".format(round(agg_metric["m_sum_MSE"], precision)))

    plot(
        target=targets[0],
        forecast=forecasts[0],
        prediction_length=dataset["metadata"].prediction_length,
    )
    plt.show()

In [9]:
# use different types of schedulers
# check different trainer_kwargs
HYPERPARAMETERS = {
    "max_epochs":            [128, 256, 512],
    "num_train_timesteps":   [50, 100, 150, 200, 250],       # explore
    "beta_start":            [1e-4],
    "beta_end":              [0.1],
    "beta_schedule":         ["linear"],                     # explore
    "context_length_coef":   [1, 2, 3, 4, 5],                # explore
    "num_layers":            [2, 3, 5],                      # explore
    "hidden_size":           [32, 64, 128],                  # explore
    "lr":                    [1e-5, 5e-5, 1e-4, 5e-4, 1e-3], # explore
    "weight_decay":          [1e-9, 1e-8, 1e-7],
    "dropout_rate":          [0.0, 0.05, 0.1, 0.15, 0.2],
    "lags_seq":              [None, [1]],                    # explore
    "num_inference_steps":   [49, 99, 149, 199, 249],
    "batch_size":            [32, 64, 128],
    "num_batches_per_epoch": [32, 64, 128],
    "num_samples":           [50, 100, 200]
}

In [ ]:
datasets = {
    "solar_nips": [],
    "electricity_nips": [],
    "exchange_rate_nips": []
}

for dataset_name in datasets:
    dataset = prepare_dataset(dataset_name)
    predictor = prepare_predictor(dataset)
    forecasts, targets, agg_metric = prepare_metrics(dataset, predictor)
    datasets[dataset_name] = {
        "dataset": dataset,
        "predictor": predictor,
        "forecasts": forecasts,
        "targets": targets,
        "agg_metric": agg_metric
    }

/opt/homebrew/anaconda3/envs/ysda_env/lib/python3.11/site-packages/gluonts/time_feature/_base.py:243: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/anaconda3/envs/ysda_env/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name  | Type          | Params | In sizes                                                             | Out sizes        
-----------------------------------------------------------------------------------------------------------------------------------
0 | model | TimeGradModel | 186 K  | [[1, 1], [1, 1], [1, 72, 5], [1, 72, 137], [1, 72, 137], [1, 24, 5]] | [1, 100, 24, 137]
----------------------------

Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 64: 'train_loss' reached 0.41120 (best 0.41120), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_1/checkpoints/epoch=0-step=64.ckpt' as top 1
Epoch 1, global step 128: 'train_loss' reached 0.31175 (best 0.31175), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_1/checkpoints/epoch=1-step=128.ckpt' as top 1
Epoch 2, global step 192: 'train_loss' reached 0.14226 (best 0.14226), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_1/checkpoints/epoch=2-step=192.ckpt' as top 1
Epoch 3, global step 256: 'train_loss' reached 0.08811 (best 0.08811), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_1/checkpoints/epoch=3-step=256.ckpt' as top 1
Epoch 4, global step 320: 'train_loss' reached 0.07767 (best 0.07767), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_1/checkpoints/epoch=4-step=320.ckpt' as top 1
Epoch 5, global step 384: 'train

Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 64: 'train_loss' reached 0.41410 (best 0.41410), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_2/checkpoints/epoch=0-step=64.ckpt' as top 1
Epoch 1, global step 128: 'train_loss' reached 0.30522 (best 0.30522), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_2/checkpoints/epoch=1-step=128.ckpt' as top 1
Epoch 2, global step 192: 'train_loss' reached 0.11737 (best 0.11737), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_2/checkpoints/epoch=2-step=192.ckpt' as top 1
Epoch 3, global step 256: 'train_loss' reached 0.07334 (best 0.07334), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_2/checkpoints/epoch=3-step=256.ckpt' as top 1
Epoch 4, global step 320: 'train_loss' reached 0.06130 (best 0.06130), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_2/checkpoints/epoch=4-step=320.ckpt' as top 1
Epoch 5, global step 384: 'train

Training: |                                               | 0/? [00:00<?, ?it/s]

Epoch 0, global step 64: 'train_loss' reached 0.41521 (best 0.41521), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_3/checkpoints/epoch=0-step=64.ckpt' as top 1
Epoch 1, global step 128: 'train_loss' reached 0.28120 (best 0.28120), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_3/checkpoints/epoch=1-step=128.ckpt' as top 1
Epoch 2, global step 192: 'train_loss' reached 0.08373 (best 0.08373), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_3/checkpoints/epoch=2-step=192.ckpt' as top 1
Epoch 3, global step 256: 'train_loss' reached 0.04662 (best 0.04662), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_3/checkpoints/epoch=3-step=256.ckpt' as top 1
Epoch 4, global step 320: 'train_loss' reached 0.03065 (best 0.03065), saving model to '/Users/npbukhanchenko/Desktop/THESIS/lightning_logs/version_3/checkpoints/epoch=4-step=320.ckpt' as top 1
Epoch 5, global step 384: 'train

In [ ]:
prepare_statistics(
    datasets["solar_nips"]["dataset"], datasets["solar_nips"]["forecasts"],
    datasets["solar_nips"]["targets"], datasets["solar_nips"]["agg_metric"]
)

In [ ]:
prepare_statistics(
    datasets["electricity_nips"]["dataset"], datasets["electricity_nips"]["forecasts"],
    datasets["electricity_nips"]["targets"], datasets["electricity_nips"]["agg_metric"]
)

In [ ]:
prepare_statistics(
    datasets["exchange_rate_nips"]["dataset"], datasets["exchange_rate_nips"]["forecasts"],
    datasets["exchange_rate_nips"]["targets"], datasets["exchange_rate_nips"]["agg_metric"]
)